In this notebook I'll demonstrate how to use the `convert_to_onnx` function to convert a pre-trained tensorflow model with either the saved model or from a frozen graph/checkpoints. This tutorial is adapted from: https://github.com/onnx/tensorflow-onnx/blob/master/tutorials/ConvertingSSDMobilenetToONNX.ipynb

**Note:** Conversion from Tensorflow requires that the `tf2onnx` package is installed. 

### Setup

In [1]:
import os
import sys

ROOT = os.getcwd()
WORK = os.path.join(ROOT, "models")
MODEL = "ssd_mobilenet_v1_coco_2018_01_28"

# force tf2onnx to cpu
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['MODEL'] = MODEL
os.environ['WORK'] = WORK

In [2]:
!cd $WORK; wget -q http://download.tensorflow.org/models/object_detection/$MODEL.tar.gz
!cd $WORK; tar zxvf $MODEL.tar.gz

x ssd_mobilenet_v1_coco_2018_01_28/
x ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.index
x ssd_mobilenet_v1_coco_2018_01_28/checkpoint
x ssd_mobilenet_v1_coco_2018_01_28/pipeline.config
x ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.data-00000-of-00001
x ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.meta
x ssd_mobilenet_v1_coco_2018_01_28/saved_model/
x ssd_mobilenet_v1_coco_2018_01_28/saved_model/saved_model.pb
x ssd_mobilenet_v1_coco_2018_01_28/saved_model/variables/
x ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb


In [3]:
!saved_model_cli show --dir $WORK/$MODEL/saved_model/ --tag_set serve  --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['inputs'] tensor_info:
      dtype: DT_UINT8
      shape: (-1, -1, -1, 3)
      name: image_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100, 4)
      name: detection_boxes:0
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100)
      name: detection_classes:0
  outputs['detection_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100)
      name: detection_scores:0
  outputs['num_detections'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: num_detections:0
Method name is: tensorflow/serving/predict


Above you can see that the model comes as a frozen graph 'frozen_inference_graph.pb' and also as a saved_model. Both can be converted to ONNX. We want to show how to convert both formats.

### Conversion to ONNX

Converting Tensorflow models to ONNX through the `convert_to_onnx` API involves slightly different inputs than conversion from the other frameworks. Instead of a model object, the `model` parameters requires a filepath to the saved model, either in a SavedModel, checkpoint, or frozen graph (.pb) format. `output_path` is a required input, and the converted ONNX model will be saved to that path. If the input file is not in the SavedModel format, then the tensorflow graph input and output names will need to be passed through `tf_input_names` and `tf_output_names` respectively. 

In [2]:
from mlisne import convert_to_onnx
import tensorflow as tf
#tf.disable_v2_behavior()

If `target_opset` is not specified, then the conversion will default to the maximum opset version supported by the user's ONNX version. With this particular SSD model from Tensorflow, however, only conversion to opset versions 10/11 are supported. Read more about versioning here: https://github.com/onnx/onnx/blob/master/docs/Versioning.md

The simplest method of conversion is with the SavedModel format, as demonstrated below.

In [3]:
convert_to_onnx(model = f"models/{MODEL}/saved_model", framework = "tensorflow",
                output_path = f"models/ssd_mobilenet.onnx")

True

The below example demonstrates conversion using the saved frozen graph file instead, which requires inputting the model input and output names as well.

In [7]:
convert_to_onnx(model = f"models/{MODEL}/frozen_inference_graph.pb", framework = "tensorflow",
                 output_path = f"models/ssd_mobilenet.onnx", tf_input_names = ["image_tensor:0"], 
                 tf_output_names = ['num_detections:0', 'detection_boxes:0', 
                                    'detection_scores:0','detection_classes:0'],
               target_opset = 10)

True

### Check conversion

We can verify that the conversion was successful by comparing the prediction results for an input image.

In [8]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import math

img = Image.open("data/ssd_image.jpg")

# we want the outputs in this order
outputs = ["num_detections:0", "detection_boxes:0", "detection_scores:0", "detection_classes:0"]

This specific model model does not require fixed image dimension so no resize of the image is needed.
But we need to reshape the flat array returned by img.getdata() to HWC and than add an additional dimension to make NHWC, aka a batch of images with 1 image in it.

In [9]:
img_data = np.array(img.getdata()).reshape(img.size[1], img.size[0], 3)
img_data = np.expand_dims(img_data.astype(np.uint8), axis=0)
print(img_data.shape)

(1, 383, 640, 3)


First we'll run the ONNX converted model

In [10]:
import onnxruntime as rt
sess = rt.InferenceSession("models/ssd_mobilenet.onnx")

In [11]:
result = sess.run(outputs, {"image_tensor:0": img_data})
onnx_num_detections, onnx_detection_boxes, onnx_detection_scores, onnx_detection_classes = result

Now we run inference on the original model and compare results

In [12]:
model = tf.saved_model.load(f"models/{MODEL}/saved_model")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [13]:
tf.compat.v1.reset_default_graph()
with tf.compat.v1.Session() as sess:
    tf.compat.v1.saved_model.load(sess, ['serve'], f"models/{MODEL}/saved_model")
    num_detections, detection_boxes, detection_scores, detection_classes = sess.run(outputs, {"image_tensor:0": img_data})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.


In [16]:
np.testing.assert_array_almost_equal(onnx_num_detections, num_detections, decimal=5)
np.testing.assert_array_almost_equal(onnx_detection_boxes, detection_boxes, decimal=5)
np.testing.assert_array_almost_equal(onnx_detection_scores, detection_scores, decimal=5)
np.testing.assert_array_almost_equal(onnx_detection_classes, detection_classes, decimal=5)